### 입력한 주소 근처에 매장이 있는지 알아보기

In [41]:
import requests
import util.item_analysis as ui
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup

# 예
market = ['로컬푸드 직매장']
addr = '울릉군 울릉읍 독도리'

# 고객이 입력한 상품을 파는 매장을 리스트로 가져오기
market_list = market

In [42]:
# 고객이 입력한 주소 ['시', '구'] 또는 ['군', '면', '리'] 가져오기
addr_list = addr.split(' ')
addr_list1 = []
addr_list2 = []
for i in addr_list:
    if i[-1] == '시':
        addr_list1.append(i)
    elif i[-1] == '구':
        addr_list1.append(i)
    elif i[-1] == '군':
        addr_list2.append(i)
    elif i[-1] == '면':
        addr_list2.append(i)
    elif i[-1] == '리':
        addr_list2.append(i)

addr1 = ' '.join(addr_list1)
addr2 = ' '.join(addr_list2)

if addr1 != '':
    addr = addr1
else:
    addr = addr2
addr

'울릉군 독도리'

In [43]:
# 고객이 입력한 주소 좌표 구하기
try:
    with open('keys/카카오api.txt') as file:
        kakao_key = file.read()

    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    url = f'{base_url}?query={quote(addr)}'
    result = requests.get(url, headers=header).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
except:
    print('주소 형식이 올바르지 않습니다.')

In [44]:
lat, lng

(37.241273540919, 131.864938493671)

In [45]:
# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)

In [46]:
# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

In [47]:
# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)

In [48]:
# '상세 주소 화살표' 클릭
try:
    juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
    juso_way[0].click()
    time.sleep(1)
except:
    print('근처에 매장이 없습니다')

근처에 매장이 없습니다


In [39]:
# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source)
market_list = soup.find('ul', class_='eDFz9')
mart = market_list.find('li', recursive=False)
mart

<li class="VLTHu" data-laim-exp-id="undefined"><div class="qbGlu"><div class="TTfa9"><a class="place_thumb" href="https://m.place.naver.com/place/1468997412/photo?entry=ple" role="button"><div class="lazyload-wrapper"><img alt="풍기인삼농협 로컬푸드직매장" data-grid-lazy="false" height="88" src="https://search.pstatic.net/common/?autoRotate=true&amp;quality=95&amp;type=f160_160&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20221230_19%2F1672387022579unS2R_JPEG%2F20221230_132142.jpg" width="88"/></div></a></div><div class="ouxiq icT4K"><a class="P7gyV" href="https://m.place.naver.com/place/1468997412?entry=ple" role="button"><div class="ApCpt"><div class="C6RjW"><span class="place_bluelink YwYLL">풍기인삼농협 로컬푸드직매장</span><span class="YzBgS">가공식품</span></div></div></a><a class="P7gyV" href="https://m.place.naver.com/place/1468997412?entry=ple" role="button"></a><div class="d7iiF"><div class="KgfA6"><span class="lWwyx NVngW"><span class="place_blind">현재 위치에서</span>1.5km</span><span class="lWwyx"><a aria-e

In [40]:
results = []
# 현재 위치에서 마켓 거리
m_distance = mart.select_one('span.lWwyx.NVngW').get_text().split('서')[1]
# 마켓 이름
m_title = mart.select_one('span.place_bluelink.YwYLL').get_text()
# 마켓 주소
# 지번만 있을 경우
if mart.select_one('div.zZfO1').get_text()[:2] == '지번':
    m_addr = mart.select_one('div.zZfO1').get_text()[2:-2]
# 도로명 주소일 경우
else:
    m_addr = mart.select_one('div.zZfO1').get_text()[3:-2]
# 딕셔너리 형태로 저장
market_data = {
    '거리': m_distance,
    '매장명': m_title,
    '주소': m_addr
}
# 딕셔너리 형태로 저장한 것을 리스트에 저장 (3가지 마켓을 넣어야 하므로)
results.append(market_data)
results

[{'거리': '1.5km', '매장명': '풍기인삼농협 로컬푸드직매장', '주소': '서울특별시 종로구 송월길 99 경희궁자이 2단지'}]